# Exploring the data
# Part 1 : Extracting the Data

In this notebook we will shortly go over the work that was done to discover the overall shape of our dataset, and how we will go about to clean it and extract what is relevant for us.

Starting an sql context.

In [1]:
from pyspark.sql import SQLContext 
sqlContext = SQLContext(sc)

We will begin by working only by working with the data of one month, to understand it and so that the computations hold on our local system.

In [3]:
text = sqlContext.read.format('com.databricks.spark.xml').options(rowTag="entity").load('02.xml')

We first look at the schema of th PySpark DataFrame so that we understand how it was loaded in our system.

In [4]:
text.printSchema()

root
 |-- full_text: string (nullable = true)
 |-- links: struct (nullable = true)
 |    |-- continuation_from: string (nullable = true)
 |    |-- continuation_to: string (nullable = true)
 |    |-- first_id: string (nullable = true)
 |    |-- last_id: string (nullable = true)
 |    |-- next_id: string (nullable = true)
 |    |-- prev_id: string (nullable = true)
 |    |-- source: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- box: string (nullable = true)
 |    |-- entity_type: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- issue_date: string (nullable = true)
 |    |-- language: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- page_no: long (nullable = true)
 |    |-- publication: string (nullable = true)
 |    |-- snp: string (nullable = true)
 |    |-- suspicious_chars_count: long (nullable = true)
 |    |-- total_chars_count: long (nullable = true)
 |    |-- updated_char_count: long (nullable = true)
 | 

And we look at the first row of our dataset.

In [5]:
text.show(1)

+--------------------+--------------------+--------------------+
|           full_text|               links|                meta|
+--------------------+--------------------+--------------------+
|YOUGOSLAVIE La re...|[null,null,Ar0010...|[87 468 733 1392,...|
+--------------------+--------------------+--------------------+
only showing top 1 row



We can also take a look at the first row as a PySpark Row type. It contains each element of each columnm we only look at the first part of the column (here the text).

In [9]:
text.first()[0][:100]

"YOUGOSLAVIE La revanche de l'histoire S'il est un pays d'Europe où la détente, avec les années, a op"

And we can look at some of the fields of the DataFrame one by one.

In [11]:
text.first().meta

Row(box='87 468 733 1392', entity_type='Article', id='Ar00100', issue_date='01/02/1989', language='French', name="YOUGOSLAVIE La revanche de l'histoire", page_no=1, publication='JDG', snp='Ar0010000.png', suspicious_chars_count=0, total_chars_count=3218, updated_char_count=3193, updated_word_count=472, word_count=691)

In [12]:
print('text.meta.box :',text.first().meta.box)
print('text.meta.snp :',text.first().meta.snp)
print('text.links.source :',text.first().links.source)

text.meta.box : 87 468 733 1392
text.meta.snp : Ar0010000.png
text.links.source : 103_JDG_1989_02_01_0001.PDF


We find that for each article, we have a text and several other parameters.

We make the first assumptions that most of these parameters will not be of real help for us so we will keep only the following parameters.

 - full_text
 - meta.issue_data As we want to know which day the article was published on
 - meta.suspicious character count We need that to know the number of characters given by the OCR reader

In [13]:
textClean = text.select('full_text','meta.issue_date','meta.suspicious_chars_count')
textClean.show(3)

+--------------------+----------+----------------------+
|           full_text|issue_date|suspicious_chars_count|
+--------------------+----------+----------------------+
|YOUGOSLAVIE La re...|01/02/1989|                     0|
|ARGENTINE Quand l...|01/02/1989|                     0|
|ARGENTINE • Suite...|01/02/1989|                     0|
+--------------------+----------+----------------------+
only showing top 3 rows



We can also look if for this month we have suspicious characters.

In [14]:
textClean.select('suspicious_chars_count').distinct().show()

+----------------------+
|suspicious_chars_count|
+----------------------+
|                     0|
+----------------------+



And we see that for january 1999, no suspicious characters, which was expected as the source is surely electronic and not paper version!

Now that we have taken a look at the data, we begin to apply functions to the data

# Part 2 : Transforming the Data

We defined a small pipeline to transform each article.
1. Separate each text into characters
2. Put each word to lower case, remove basic stopwords (. , "'" etc..)
3. Remove common words that are not useful to our analysis (le, la, de, te etc..)
4. Count the number of times each resulting words, and how many words are in total (needed for word frequency).

Let's take a look at the processing steps for one article.

In [21]:
article1 = text.select('full_text').first().full_text

In [24]:
sc.parallelize([article1]).map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower())\
                          .flatMap(lambda x: x.split(' '))\
                          .map(lambda x: (x, 1))\
                          .reduceByKey(lambda x,y:x+y)\
                          .map(lambda x:(x[1],x[0]))\
                          .sortByKey(False).take(10)

[(84, ''),
 (32, 'de'),
 (26, 'la'),
 (18, 'les'),
 (12, 'et'),
 (11, 'le'),
 (10, 'que'),
 (10, 'des'),
 (10, 'à'),
 (7, 'du')]

We can see here that we still have to do some cleaning but we have been able to count how many times a word occurs in an article.

The aim now was to apply this pipeline to the whole dataset that was stored on the cluster, but we were unable to make that work due to errors that would have taken too much time to solve.

We therefore decided to download the entire dataset on our personal computers and work locally only with Python, without Spark.
The next step of our project is described in the subsequent notebooks.